In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import rdkit
import re
import os
import pickle
from rdkit import Chem
from posebusters import PoseBusters
import json
import torch
from tqdm import tqdm
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
# This enables inline rendering of molecules
IPythonConsole.ipython_useSVG=True 

In [ ]:
cart_1x_path = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B_conformers/c037e75255bc41c19c716939/step-4500"
cart_2x_path = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B_conformers/d267db61f57d4b428baa604a/step-9000"
cart_4x_path = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B_conformers/3408e9758572478c80393771/step-18000"
cart_6e_path = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B_conformers/301b8328481243c6aa8d8003/step-27000"
cart_8e_path = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B_conformers/c13311b27056459eaccf5877/step-36000"
m100_100 = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-27M_conformers/afebcc510dec403f9532dff6/step-42600"
m100_120p_path = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-100M_conformers/c29bd453f4ff497d8c99c8f7/step-30000"
new_model = "/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-170M_conformers/723a4ada660e4227b29a904a/step-10000"

tokenizer  = AutoTokenizer.from_pretrained("/auto/home/menuab/code/YNNtitan/torchtitan/tokenizers/Llama-3.2-chem-1B-v1", padding_side='left')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(new_model, 
                                             torch_dtype=torch.float32).to("cuda:0")
model.generation_config.pad_token_id = tokenizer.pad_token_id
model.dtype, model.device

ValueError: Unrecognized configuration class <class 'transformers.models.deprecated.van.configuration_van.VanConfig'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, ElectraConfig, ErnieConfig, FalconConfig, FalconMambaConfig, FuyuConfig, GemmaConfig, Gemma2Config, GitConfig, GlmConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, GraniteConfig, GraniteMoeConfig, JambaConfig, JetMoeConfig, LlamaConfig, MambaConfig, Mamba2Config, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MllamaConfig, MoshiConfig, MptConfig, MusicgenConfig, MusicgenMelodyConfig, MvpConfig, NemotronConfig, OlmoConfig, Olmo

: 

In [ ]:
import transformers
top_p_sampling_config = transformers.GenerationConfig (
  do_sample=True,
  temperature=0.8,
  top_p=0.9
)

In [42]:
pad_len = 0
canonical_smiles = 'O=C(Cn1cccc1C(=O)c1ccccc1)NCCN1CCOCC1'
canonical_smiles = 'O=C(Cc1c[nH]c2ccccc12)NNC(=O)c1ccccc1'
prompt = [f"[SMILES]{canonical_smiles}[/SMILES]"]
# prompt = "[SMILES]"
prompt = tokenizer(prompt, 
                   padding=True, 
                #    max_length=len(tokenizer(prompt)["input_ids"])+pad_len, 
                   return_tensors="pt", 
                   add_special_tokens=True).to(model.device)
output = model.generate(input_ids=prompt["input_ids"], 
                        attention_mask=prompt["attention_mask"], 
                        max_new_tokens=3000, 
                        eos_token_id=128329, 
                        num_return_sequences=200,
                        generation_config=top_p_sampling_config)
output_ = tokenizer.batch_decode(output)


In [43]:
max(len(o) for o in output_)

669

In [44]:
prompt['input_ids'].shape

torch.Size([1, 27])

In [45]:
tokenizer.batch_decode(output)

['<|begin_of_text|>[SMILES]O=C(Cc1c[nH]c2ccccc12)NNC(=O)c1ccccc1[/SMILES][CONFORMER]O<0.1491,2.9834,-0.1071>=C<-0.6373,2.0955,0.1629>(C<-2.1198,2.1715,-0.1708>c<-2.8551,0.8733,-0.1908>1c<-4.0600,0.6197,0.4007>[nH]<-4.4443,-0.6699,0.1576>c<-3.4925,-1.2829,-0.6087>2c<-3.4269,-2.5809,-1.0993>c<-2.3162,-2.9224,-1.8380>c<-1.3084,-1.9931,-2.1000>c<-1.3748,-0.7029,-1.6230>c<-2.4735,-0.3312,-0.8450>12)N<-0.2645,1.0176,0.8816>N<1.0635,0.6660,1.0813>C<1.7623,0.0108,0.0984>(=O<1.2408,-0.3495,-0.9394>)c<3.1970,-0.2078,0.4153>1c<3.9317,0.6511,1.2211>c<5.2707,0.4033,1.4564>c<5.8874,-0.7038,0.9036>c<5.1644,-1.5739,0.0989>c<3.8268,-1.3285,-0.1450>1[/CONFORMER]',
 '<|begin_of_text|>[SMILES]O=C(Cc1c[nH]c2ccccc12)NNC(=O)c1ccccc1[/SMILES][CONFORMER]O<-1.1174,-3.0160,0.1176>=C<-0.2362,-2.2150,0.3610>(C<1.2321,-2.5547,0.5354>c<2.1109,-1.5635,-0.1534>1c<2.4359,-1.5457,-1.4786>[nH]<3.2136,-0.4534,-1.7551>c<3.4086,0.2655,-0.6100>2c<4.1196,1.4397,-0.3880>c<4.1410,1.9427,0.8929>c<3.4811,1.2960,1.9448>c<2.7748,0.

In [4]:
gen_path = "/auto/home/menuab/code/3DMolGen/gen_results/2025-05-14-14:13_new1b/generation_resutls.pickle"
def load_pkl(file_path: str):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist.")
    with open(file_path, "rb") as f:
        return pickle.load(f)
# gen_results = load_pkl(gen_path)

In [5]:
ref_data = load_pkl("/auto/home/menuab/code/3DMolGen/drugs_test_inference_.pickle")    

In [6]:
ref_data['NC(=O)C1CCC1']

{'geom_smiles': 'NC(=O)C1CCC1',
 'geom_smiles_c': 'NC(=O)C1CCC1',
 'confs': [<rdkit.Chem.rdchem.Mol at 0x766934290540>,
 'num_confs': 2,
 'pickle_path': 'drugs/NC(=O)C1CCC1.pickle',
 'canonical_smiles': 'NC(=O)C1CCC1'}

In [17]:
for k,v in gen_results.items():
    print(k,'----')
    for c in v:
        print(Chem.MolToSmiles(c, canonical=True, isomericSmiles=True))

NC(=O)C1CCC1 ----
NC(=O)C1CCC1
NC(=O)C1CCC1
NC(=O)C1CCC1
NC(=O)C1CCC1
C[NH2+]Cc1sccc1C ----
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
C[NH2+]Cc1sccc1C
Cc1ccccc1CNc1nnnn1C ----
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1nnnn1C
Cc1ccccc1CNc1

In [14]:
canonical_smiles = 'CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21'
prompt = [f"[SMILES]{canonical_smiles}[/SMILES]"] * 10
prompt

['[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]',
 '[SMILES]CN1c2ccccc2Oc2nnc(N3CCN(Cc4ccccc4)CC3)cc21[/SMILES]']

In [ ]:
# file = open('drugs_test_mols_inference.jsonl','w') 
ref, gen, incs = [], [], []
inc = 0
import re
for en, mol_dict in enumerate(tqdm(test_mols)):
    canonical_smiles = mol_dict["canonical_smiles"]
    canonical_smiles = '[H][O][C]1=[C]([C](=[N][C]([H])([H])[C]([H])([H])[c]2[c]([H])[c]([H])[c]([H])[c]([H])[c]2[H])[C]([H])([H])[H])[C](=[O])[S][C]1([H])[H]'

    geom_smiles = mol_dict["geom_smiles"]
    num_generations = mol_dict["num_confs"] * 2
    print(f"mol num: {en+1} generating {num_generations} conformers for {geom_smiles}")    
    generations = []
    prompt = f"[SMILES]{canonical_smiles}[/SMILES]"
    prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).to(model.device).input_ids
    
    # output = model.generate(prompt, max_new_tokens=2000, eos_token_id=128329, do_sample=False)
    output = model.generate(prompt, max_length=2000, eos_token_id=128329, 
                        # num_beams=10,
                        # num_beam_groups=2,
                        # diversity_penalty=1.1,
                        num_return_sequences=2, 
                        do_sample=True,
                        top_p=0.90,
                        temperature=0.8,
                        # top_k=3
                        )
    print(f"len prompt toks: {len(prompt[0])}, len gen toks: {len(output[0])-len(prompt[0])}")
    output = tokenizer.batch_decode(output)
    # print("raw output: ", output)
    # print("canonical_smiles: ", canonical_smiles)
    # display(Chem.MolFromSmiles(canonical_smiles))
    for out in output:
        generated_conformer = out[out.find("[CONFORMER]")+len("[CONFORMER]"):out.find("[/CONFORMER]")]
        # print(generated_conformer)
        generated_smiles = re.sub(r'<[^>]*>', '', generated_conformer)
        if generated_smiles == canonical_smiles:
            ref.append(geom_smiles)
            gen.append(generated_smiles)
            print(f"{canonical_smiles=}")
            print(f"{generated_smiles=}")
            print(f"{out=}")
            sample = {
                "geom_smiles": geom_smiles,
                "generated_conformer": generated_conformer
            }
            file.write(f"{json.dumps(sample)}\n")
        else:
            print("smiles didn't match for ")
            print(f"{canonical_smiles=}")
            print(f"{generated_smiles=}")
            print(out)
            incs.append(out)
            inc += 1
        # print("generated_smiles: ", generated_smiles)
        # display(Chem.MolFromSmiles(generated_smiles))
    print('----------------------')
    file.close()
    break
    # if en==100:
    #     break
    

  0%|          | 0/1000 [00:00<?, ?it/s]

mol num: 1 generating 138 conformers for C#CCNC(=O)C1=C[C@@H](c2ccc(Br)cc2)C[C@@H](OCc2ccc(CO)cc2)O1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
  0%|          | 0/1000 [00:11<?, ?it/s]

len prompt toks: 88, len gen toks: 602
canonical_smiles='[H][O][C]1=[C]([C](=[N][C]([H])([H])[C]([H])([H])[c]2[c]([H])[c]([H])[c]([H])[c]([H])[c]2[H])[C]([H])([H])[H])[C](=[O])[S][C]1([H])[H]'
generated_smiles='[H][O][C]1=[C]([C](=[N][C]([H])([H])[C]([H])([H])[c]2[c]([H])[c]([H])[c]([H])[c]([H])[c]2[H])[C]([H])([H])[H])[C](=[O])[S][C]1([H])[H]'
out='<|begin_of_text|>[SMILES][H][O][C]1=[C]([C](=[N][C]([H])([H])[C]([H])([H])[c]2[c]([H])[c]([H])[c]([H])[c]([H])[c]2[H])[C]([H])([H])[H])[C](=[O])[S][C]1([H])[H][/SMILES][CONFORMER][H<-2.1227,-2.4066,0.0171>][O<-3.0615,-2.0513,0.0322>][C<-2.9815,-0.7348,0.0014>]1=[C<-1.9039,0.0974,-0.0384>]([C<-0.5432,-0.4122,-0.0478>](=[N<0.4296,0.4126,-0.1234>][C<1.8135,0.0420,-0.1334>]([H<1.9969,-0.8204,-0.7864>])([H<2.1302,-0.2397,0.8801>])[C<2.6484,1.2332,-0.6214>]([H<2.4324,1.4031,-1.6782>])([H<2.3361,2.1208,-0.0685>])[c<4.1106,0.9633,-0.4145>]2[c<4.7994,0.1287,-1.2863>]([H<4.2861,-0.2955,-2.1375>])[c<6.1366,-0.1581,-1.0765>]([H<6.6606,-0.8065,-1.7634>]

In [6]:
m_gen = '[H<-0.0541,-1.9089,0.0001>][O<-0.9993,-2.2537,0.0001>][C<-1.7999,-1.2063,0.0001>]1=[C<-1.4564,0.1106,0.0001>]([C<-0.0788,0.5689,0.0001>](=[N<0.8417,-0.3277,0.0001>][C<2.2485,-0.0474,0.0001>]([H<2.5225,0.5473,0.8810>])([H<2.5319,0.5279,-0.8907>])[C<3.0100,-1.3801,0.0118>]([H<2.7208,-1.9349,0.9052>])([H<2.7059,-1.9615,-0.8595>])[c<4.4899,-1.1329,0.0004>]2[c<5.2423,-1.3378,1.1489>]([H<4.7600,-1.7082,2.0424>])[c<6.6015,-1.0774,1.1557>]([H<7.1753,-1.2451,2.0552>])[c<7.2245,-0.6015,0.0147>]([H<8.2852,-0.3985,0.0200>])[c<6.4818,-0.3901,-1.1344>]([H<6.9632,-0.0206,-2.0281>])[c<5.1237,-0.6519,-1.1403>]2[H<4.5471,-0.4887,-2.0396>])[C<0.1665,2.0521,-0.0001>]([H<1.2285,2.2721,-0.0001>])([H<-0.2910,2.4999,-0.8815>])[H<-0.2959,2.5005,0.8790>])[C<-2.6091,1.0094,0.0001>](=[O<-2.6401,2.2087,0.0001>])[S<-4.1199,0.0224,-0.0001>][C<-3.2319,-1.5780,-0.0001>]1([H<-3.5528,-2.1773,-0.8520>])[H<-3.4629,-2.1250,0.9149>]'